# TFM. Modelo predictivo llamadas Call Center
## Notebook 1 - Importación datos de envíos

## Importacion de librerias 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import datetime
import calendar

pd.options.display.max_columns = None

### Importacion de los datos de envios por oficinas

In [2]:
df = pd.read_csv('data/shippings.csv', low_memory=False)

In [3]:
df['Franquicia destino'].unique()

array([4800, 4802, 4803, 4806, 4810], dtype=int64)

In [4]:
#df = df[ (df_shippings['Franquicia destino']==4800) ]

In [5]:
df.columns

Index(['Unnamed: 0', 'Fecha envío', 'Número envío', 'Id. Fiscal',
       'Nombre Comercial', 'Código servicio', 'Nombre Rem', 'Población Rem',
       'CP Rem', 'Nombre vía Rem', 'Nombre', 'Población', 'Código postal',
       'Nombre vía', 'Total bultos', 'Franquicia origen', 'Franquicia destino',
       'Total', 'Estado', 'Tipo anomalía', 'Motivo', 'Importe Total'],
      dtype='object')

In [6]:
df = df.iloc[:, [1,5,16]]
df.head()

,Fecha envío,Código servicio,Franquicia destino
0,01/01/2019,Económico,4800
1,01/01/2019,Marítimo baleares,4800
2,01/01/2019,E-commerce,4800
3,01/01/2019,E-commerce,4800
4,01/01/2019,E-commerce,4800


In [7]:
df['Fecha envío'] = pd.to_datetime(df['Fecha envío'])

In [8]:
df[df['Fecha envío'].isnull()]

,Fecha envío,Código servicio,Franquicia destino


In [9]:
dias = list(calendar.day_name)

In [10]:
df['dayofweek'] = [dias[numero] for numero in df['Fecha envío'].dt.dayofweek]
df.head()

,Fecha envío,Código servicio,Franquicia destino,dayofweek
0,2019-01-01,Económico,4800,Tuesday
1,2019-01-01,Marítimo baleares,4800,Tuesday
2,2019-01-01,E-commerce,4800,Tuesday
3,2019-01-01,E-commerce,4800,Tuesday
4,2019-01-01,E-commerce,4800,Tuesday


In [11]:
df['Fecha envío'] =  pd.to_datetime(df['Fecha envío']).astype('str')

In [12]:
df['Código servicio'].unique()

array(['Económico', 'Marítimo baleares', 'E-commerce', 'Urgente 12',
       'Urgente 10', 'Mascotas', 'Urgente 14', 'Urgente 8:30',
       'Urgente 12 expedición', 'Burofax', 'Urgente 19 expedición',
       'Urgente hoy', nan, 'Documentos 14', 'Urgente 8:30 expedición',
       'Urgente 10 expedición', 'Urgente 19', 'Mrw economy (ecopaq)',
       'Bag 19', 'Retorno albarán', 'E-commerce canje',
       'Express documentos', 'Bag 14', 'Retorno canje',
       'Urgente 14 expedición', 'Devolución burofax',
       'OperativaServicioEnumECOBA', 'OperativaServicioEnumU19E;U19;E',
       'Urgente funchal', 'Express paquetes', 'E-Commerce', 'Plan ayuda',
       'Ecobox 25', 'Retorno burofax'], dtype=object)

In [13]:
df = df[['Fecha envío','Código servicio','dayofweek']]
df.reset_index(inplace=True)
df = df[['Fecha envío','Código servicio','dayofweek']]
df['total'] = 1

In [14]:
df.head()

,Fecha envío,Código servicio,dayofweek,total
0,2019-01-01,Económico,Tuesday,1
1,2019-01-01,Marítimo baleares,Tuesday,1
2,2019-01-01,E-commerce,Tuesday,1
3,2019-01-01,E-commerce,Tuesday,1
4,2019-01-01,E-commerce,Tuesday,1


In [15]:
encoder = OneHotEncoder(handle_unknown='ignore')
enc = df[['Código servicio']]

In [16]:
encoder.fit(enc.values.reshape(-1,1))
encoder.categories_

[array(['Bag 14', 'Bag 19', 'Burofax', 'Devolución burofax',
        'Documentos 14', 'E-Commerce', 'E-commerce', 'E-commerce canje',
        'Ecobox 25', 'Económico', 'Express documentos', 'Express paquetes',
        'Marítimo baleares', 'Mascotas', 'Mrw economy (ecopaq)',
        'OperativaServicioEnumECOBA', 'OperativaServicioEnumU19E;U19;E',
        'Plan ayuda', 'Retorno albarán', 'Retorno burofax',
        'Retorno canje', 'Urgente 10', 'Urgente 10 expedición',
        'Urgente 12', 'Urgente 12 expedición', 'Urgente 14',
        'Urgente 14 expedición', 'Urgente 19', 'Urgente 19 expedición',
        'Urgente 8:30', 'Urgente 8:30 expedición', 'Urgente funchal',
        'Urgente hoy', nan], dtype=object)]

In [17]:
encoded = encoder.transform(enc.values.reshape(-1,1))
encoded_df = pd.DataFrame(encoded.todense())
encoded_df.columns = encoder.categories_[0]

In [18]:
df = pd.concat([df,encoded_df], axis=1)

In [19]:
#encoder = OneHotEncoder(handle_unknown='ignore')
enc = df[['dayofweek']]
encoder.fit(enc.values.reshape(-1,1))
encoded = encoder.transform(enc.values.reshape(-1,1))
encoded_df = pd.DataFrame(encoded.todense())
encoded_df.columns = encoder.categories_[0]

In [20]:
df = pd.concat([df,encoded_df], axis=1)

In [21]:
#encoder = OneHotEncoder(handle_unknown='ignore')
#enc = train[['Código postal']]
#encoder.fit(enc.values.reshape(-1,1))
#encoded = encoder.transform(enc.values.reshape(-1,1))
#encoded_df = pd.DataFrame(encoded.todense())
#encoded_df.columns = encoder.categories_[0]

In [22]:
#train = pd.concat([train,encoded_df], axis=1)

In [23]:
df.drop(columns=['Código servicio','dayofweek'], inplace=True)

In [24]:
df = df.groupby(['Fecha envío']).sum()

In [25]:
df.head()

,total,Bag 14,Bag 19,Burofax,Devolución burofax,Documentos 14,E-Commerce,E-commerce,E-commerce canje,Ecobox 25,Económico,Express documentos,Express paquetes,Marítimo baleares,Mascotas,Mrw economy (ecopaq),OperativaServicioEnumECOBA,OperativaServicioEnumU19E;U19;E,Plan ayuda,Retorno albarán,Retorno burofax,Retorno canje,Urgente 10,Urgente 10 expedición,Urgente 12,Urgente 12 expedición,Urgente 14,Urgente 14 expedición,Urgente 19,Urgente 19 expedición,Urgente 8:30,Urgente 8:30 expedición,Urgente funchal,Urgente hoy,NaN,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
Fecha envío,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-01,33,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,2.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,0.0
2019-01-02,1923,0.0,5.0,14.0,0.0,5.0,0.0,424.0,0.0,0.0,311.0,0.0,0.0,723.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,127.0,0.0,187.0,10.0,13.0,0.0,50.0,38.0,5.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1923.0
2019-01-03,1408,0.0,0.0,12.0,0.0,3.0,0.0,348.0,0.0,0.0,208.0,0.0,0.0,540.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,80.0,4.0,131.0,4.0,5.0,0.0,1.0,42.0,3.0,1.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,1408.0,0.0,0.0
2019-01-04,3220,0.0,1.0,10.0,0.0,3.0,0.0,888.0,1.0,0.0,442.0,0.0,0.0,1183.0,25.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,176.0,7.0,286.0,14.0,3.0,0.0,104.0,69.0,5.0,1.0,0.0,0.0,0.0,3220.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-05,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0


In [26]:
df.columns

Index([                          'total',                          'Bag 14',
                                'Bag 19',                         'Burofax',
                    'Devolución burofax',                   'Documentos 14',
                            'E-Commerce',                      'E-commerce',
                      'E-commerce canje',                       'Ecobox 25',
                             'Económico',              'Express documentos',
                      'Express paquetes',               'Marítimo baleares',
                              'Mascotas',            'Mrw economy (ecopaq)',
            'OperativaServicioEnumECOBA', 'OperativaServicioEnumU19E;U19;E',
                            'Plan ayuda',                 'Retorno albarán',
                       'Retorno burofax',                   'Retorno canje',
                            'Urgente 10',           'Urgente 10 expedición',
                            'Urgente 12',           'Urgente 12 expedición',

In [27]:
df.describe().iloc[0,0] ##Cantidad de registros

851.0

In [28]:
'''columnas = ['E-commerce','Económico','Marítimo baleares',\
            'Urgente 10','Urgente 12','Urgente 19','Urgente 19 expedición',\
            'Monday','Tuesday','Wednesday','Thursday','Friday']'''

"columnas = ['E-commerce','Económico','Marítimo baleares',            'Urgente 10','Urgente 12','Urgente 19','Urgente 19 expedición',            'Monday','Tuesday','Wednesday','Thursday','Friday']"

In [29]:
#df = df[columnas]
#df.head()

In [30]:
#df = df.iloc[:,1:-7]

In [31]:
#df.head()

In [32]:

df.Monday = [ 1 if value > 0 else 0 for value in df.Monday]
df.Tuesday = [ 1 if value > 0 else 0 for value in df.Tuesday]
df.Wednesday = [ 1 if value > 0 else 0 for value in df.Wednesday]
df.Thursday = [ 1 if value > 0 else 0 for value in df.Thursday]
df.Friday = [ 1 if value > 0 else 0 for value in df.Friday]
df.Saturday = [ 1 if value > 0 else 0 for value in df.Saturday]
df.Sunday = [ 1 if value > 0 else 0 for value in df.Sunday]
df.head()


,total,Bag 14,Bag 19,Burofax,Devolución burofax,Documentos 14,E-Commerce,E-commerce,E-commerce canje,Ecobox 25,Económico,Express documentos,Express paquetes,Marítimo baleares,Mascotas,Mrw economy (ecopaq),OperativaServicioEnumECOBA,OperativaServicioEnumU19E;U19;E,Plan ayuda,Retorno albarán,Retorno burofax,Retorno canje,Urgente 10,Urgente 10 expedición,Urgente 12,Urgente 12 expedición,Urgente 14,Urgente 14 expedición,Urgente 19,Urgente 19 expedición,Urgente 8:30,Urgente 8:30 expedición,Urgente funchal,Urgente hoy,NaN,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
Fecha envío,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-01,33,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,2.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0
2019-01-02,1923,0.0,5.0,14.0,0.0,5.0,0.0,424.0,0.0,0.0,311.0,0.0,0.0,723.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,127.0,0.0,187.0,10.0,13.0,0.0,50.0,38.0,5.0,0.0,0.0,0.0,7.0,0,0,0,0,0,0,1
2019-01-03,1408,0.0,0.0,12.0,0.0,3.0,0.0,348.0,0.0,0.0,208.0,0.0,0.0,540.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,80.0,4.0,131.0,4.0,5.0,0.0,1.0,42.0,3.0,1.0,0.0,0.0,23.0,0,0,0,0,1,0,0
2019-01-04,3220,0.0,1.0,10.0,0.0,3.0,0.0,888.0,1.0,0.0,442.0,0.0,0.0,1183.0,25.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,176.0,7.0,286.0,14.0,3.0,0.0,104.0,69.0,5.0,1.0,0.0,0.0,0.0,1,0,0,0,0,0,0
2019-01-05,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0


In [33]:
df = df.drop(labels=['Saturday','Sunday'], axis=1)

In [34]:
df.to_pickle('./data/df_Envios')